In [1]:
import numpy as np
import sys
import time
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

sys.path.extend(['/Users/KatherineAnardeWheels/PycharmProjects/Barrier3D', '/Users/KatherineAnardeWheels/PycharmProjects/brie', '/Users/KatherineAnardeWheels/PycharmProjects/CASCADE'])
datadir = "../B3D_Inputs/"  # laptop 
Path(datadir)
#datadir = "/Users/KatherineAnardeWheels/PycharmProjects/CASCADE/B3D_Inputs/"  # laptop
notebook_dir = "../notebooks/"

from cascade import Cascade
from scripts import CASCADE_plotters as CASCADEplt

In [2]:
alongshore_section_count = 6
background_erosion = [-1] * alongshore_section_count
nt = 100
roadway_management_on = [True]*3 + [False]*3
beach_nourishment_on = [False]*3 + [True]*3

cascade = Cascade(
    datadir,
    name="AST_demonstration",
    wave_height=1,
    wave_period=7,
    wave_asymmetry=0.8,
    wave_angle_high_fraction=0.2,
    sea_level_rise_rate=0.004,
    sea_level_rise_constant=True,
    background_erosion=background_erosion,
    alongshore_section_count=alongshore_section_count,
    time_step_count=nt,
    num_cores=1,
    roadway_management_module=roadway_management_on,  # no roadway management
    alongshore_transport_module=True,  # couple brie        
    beach_nourishment_module=beach_nourishment_on,
    community_dynamics_module=False,  # no community dynamics
    nourishment_interval=10,  # yrs
    nourishment_volume=100,  # m^3/m
    overwash_filter=90,  # % overwash removed
    overwash_to_dune=10, 
)

# Loop for 50 years at a 10 year interval, 100 m^3/m and then 50 years at a 20 year interval with 300 m^3/m
nt = 50
for _ in tqdm(range(nt-1)):
    cascade.update()
    if cascade.b3d_break:
        break
        
# # during the CASCADE initialization, the nourishment interval and volume is specified individually for each 
# # barrier3d alongshore cell; so to update these values, we need to specify which barrier3d cell we want to modify
# # (here, we only have one cell)
# for 
# cascade.nourishment_interval[iB3D] = 20  # increase to 20 years
# cascade.nourishment_volume[iB3D] = 300  # increase to 300 m^3/m

# for _ in tqdm(range(nt)):
#     cascade.update()
#     if cascade.road_break or cascade.b3d_break:
#         break

100%|██████████| 49/49 [02:05<00:00,  2.56s/it]
